<a href="https://colab.research.google.com/github/AlexKressner/OR_Workshop/blob/main/Ganzzahlige_Optimierung_Spediteur.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Auswahlproblem Spediteur

In [ ]:
# Notwendigen Programminstallationen
# pip als Paketmanager
!pip install -U -q pip
!pip install -q ortools
# Laden der Packages
import pandas as pd
from ortools.linear_solver import pywraplp
from google.colab import drive

In [ ]:
# Solver mit SCIP als Backend.
# SCIP implementiert Simplex, Branch-and-Bound, etc
solver = pywraplp.Solver.CreateSolver('SCIP')

## Daten aus Google Drive laden


1.   Problemrelevante Daten in Google-Drive laden
2.   Google-Drive mit Colab-Notebook verbinden
3.   Daten mit `pandas` laden

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Ordner finden
! ls drive/MyDrive/OR_Workshop/Daten

MasterPlanning_Beschaffungskapazitäten.csv
MasterPlanning_Beschaffungskosten.csv
MasterPlanning_Direktbedarfskoeffizienten.csv
MasterPlanning_Lagerkosten.csv
MasterPlanning_Nachfrage.csv
MasterPlanning_Produktionskapazitäten.csv
MasterPlanning_Produktionskosten.csv
MasterPlanning_Transportkosten_Beschaffung.csv
MasterPlanning_Transportkosten_Distribution.csv
NetworkDesign_Beschaffungskosten.csv
NetworkDesign_Invest_Kapa.csv
NetworkDesign_Nachfrage.csv
NetworkDesign_Produktionskosten.csv
Spediteur_Auftraege.csv
Spediteur_LKW.csv


In [ ]:
# Pfad zurückgeben
! cd drive/MyDrive/OR_Workshop/Daten && pwd

/content/drive/MyDrive/OR_Workshop/Daten


In [ ]:
path = "/content/drive/MyDrive/OR_Workshop/Daten"

In [ ]:
orders = pd.read_csv(f"{path}/Spediteur_Auftraege.csv", sep=";", decimal=",")

In [ ]:
orders

,Auftrag,Deckungsbeitrag,Volumen
0,A1,10,3.00
1,A2,15,4.50
2,A3,25,9.25
3,A4,5,1.20
4,A5,11,4.00
5,A6,8,2.70


## Indexmengen

In [ ]:
J = orders["Auftrag"].unique().tolist() # Menge der Aufträge

In [ ]:
J

['A1', 'A2', 'A3', 'A4', 'A5', 'A6']

## Parameter

In [ ]:
c = orders.set_index(["Auftrag"]).to_dict("dict")["Deckungsbeitrag"]
c

{'A1': 10, 'A2': 15, 'A3': 25, 'A4': 5, 'A5': 11, 'A6': 8}

In [ ]:
a = orders.set_index(["Auftrag"]).to_dict("dict")["Volumen"]
a

{'A1': 3.0, 'A2': 4.5, 'A3': 9.25, 'A4': 1.2, 'A5': 4.0, 'A6': 2.7}

In [ ]:
b = 10 # Kapazitäten LKW

## Entscheidungsvariablen

In [ ]:
# Definition der Entscheidungsvariablen
x={}
for j in J: 
  x[j] = solver.BoolVar(j)
print('Anzahl Entscheidungsvariablen =', solver.NumVariables())

Anzahl Entscheidungsvariablen = 6


## Zielfunktion

In [ ]:
# Maximierung der gesamten Deckungsbeiträge
solver.Maximize(sum(c[j]*x[j] for j in J))

## Nebenbedingungen

In [ ]:
solver.Add(sum(a[j]*x[j] for j in J)<=b)

<ortools.linear_solver.pywraplp.Constraint; proxy of <Swig Object of type 'operations_research::MPConstraint *' at 0x7ff30f1a0cf0> >

## Berechnung Lösung

In [ ]:
status = solver.Solve()

if status == pywraplp.Solver.OPTIMAL:
    print('LÖSUNG:')
    print('Zielfunktionswert (Deckungsbeitrag) =', solver.Objective().Value())
    for j in J:
        print(f'{j} =', round(x[j].solution_value()))
else:
    print('Problem hat keine Lösung')

LÖSUNG:
Zielfunktionswert (Deckungsbeitrag) = 31.000000000000004
A1 = 0
A2 = 1
A3 = 0
A4 = 1
A5 = 1
A6 = 0
